In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
%matplotlib inline
from keras import models
import numpy as np
import shutil
import pandas as pd
import glob
import matplotlib.image as mpimg

from keras import backend as K

In [ ]:
def plot_heatmap(row, predicted_label, model, plot_ix=None):
    
    if plot_ix is None:
        ix = np.argmax(predicted_label)
        output_file = 'estimated.jpg'
    else:
        if plot_ix:
            output_file = 'damage.jpg'
        else:
            output_file = 'other.jpg'
            
        ix = plot_ix
        
    image_output = model.output[:, ix]
    # The is the output feature map of the `block5_conv4` layer,
    # the last convolutional layer in VGG19
    last_conv_layer = model.get_layer('vgg19')

    # This is the gradient of the estimated class with regard to
    # the output feature map of the vgg19 model
    grads = K.gradients(image_output, last_conv_layer.get_output_at(1))[0]
    
    # This is a vector of shape (512,), where each entry
    # is the mean intensity of the gradient over a specific feature map channel
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
    # This function allows us to access the values of the quantities we just defined:
    # `pooled_grads` and the output feature map of `block5_conv4`,
    # given a sample image
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.get_output_at(1)])

    # These are the values of these two quantities, as Numpy arrays,
    # given our sample image of two elephants
    pooled_grads_value, conv_layer_output_value = iterate([image_array])

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the elephant class
    for i in range(512):
        conv_layer_output_value[0, :, :, i] *= pooled_grads_value[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.nanmean(conv_layer_output_value, axis=-1)
    
    heatmap = np.maximum(heatmap[0], 0)
    heatmap /= np.max(heatmap)
    if np.isnan(np.max(heatmap)):
        return 0
    # We use cv2 to load the original image
    img = cv2.imread(image_path + row['folder'] + '/' + image)

    # We resize the heatmap to have the same size as the original image
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # We convert the heatmap to RGB
    heatmap = np.uint8(255 * heatmap)

    # We apply the heatmap to the original image
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # 0.4 here is a heatmap intensity factor
    superimposed_img = heatmap * 0.4 + img

    # Save the image to disk
    cv2.imwrite(output_file, superimposed_img)
    return 1

In [ ]:
def make_double_plot(image_list, titles, filename=None):
    rows = 1
    cols = 2
    axes=[]

    fig=plt.figure(figsize=(18,12))

    for i, a in enumerate(range(rows*cols)):
        img = image_list[i]
        axes.append( fig.add_subplot(rows, cols, a+1) )
        subplot_title=titles[i]
        axes[-1].set_title(subplot_title)  
        plt.imshow(img)
    fig.tight_layout()   
    if filename:
        plt.savefig(filename, bbox_inches = 'tight',  pad_inches = 0.1)
        plt.close()
    else:
        plt.show()
    

In [ ]:
model = models.load_model(
    './trained_models/2_class_VGG19/Best_model.h5')

In [ ]:
image_path = '../dataset027/'

label = {0:'non-damge', 1:'damage'}

In [ ]:
li = []
for item in glob.glob('../../dataset027/output_*'):
    filename = item + '/output.csv'
    folder = item.split('_')[1]
    df_tmp = pd.read_csv(filename, index_col=None, header=0)
    df_tmp['folder'] = folder
    li.append(df_tmp)

output_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
for ix, row in output_df.iterrows():
    image = row['image']
    folder = row['folder']
    img_temp = cv2.resize(cv2.imread(image_path+folder+'/'+image, cv2.IMREAD_COLOR), (150, 150), interpolation=cv2.INTER_CUBIC)
    b,g,r = cv2.split(img_temp)           # get b, g, r
    rgb_img1 = cv2.merge([r,g,b])     # switch it to r, g, b
    rgb_img1
    X_any = []
    X_any.append(rgb_img1)
    image_array = np.array(X_any)/255
    preds = model.predict(image_array)
    predicted_label = preds.flatten()
        
    a = plot_heatmap(row, predicted_label, model, plot_ix=0)
    if not a:
        continue
    superimposed_img = mpimg.imread('./other.jpg')
    
    a = plot_heatmap(row, predicted_label, model, plot_ix=1)
    if not a:
        continue
    superimposed_img2 = mpimg.imread('./damage.jpg')
    
    image_list = [superimposed_img, superimposed_img2]
    titles = ['Ground truth: %s, estimated: %s\n with probability %.2f'%(label[row['groundTruth']], 'non-damage', 1 - row['output']), 
              'Ground truth: %s, estimated: %s\n with probability %.2f'%(label[row['groundTruth']], 'damage', row['output'])]

    output_name = './grad_cam_results/' + image
    make_double_plot(image_list, titles,output_name)